In [47]:


import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer  # Til tekstbehandling
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, f1_score



In [48]:
df=pd.read_csv('cleanedStemmedCATbbc2.csv') 

In [49]:
df['broad_category'].value_counts()

broad_category
Fake News        43590
Reliable News    41247
Name: count, dtype: int64

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['broad_category'], test_size=0.2, random_state=42)

X_train_vec og X_test_vec indeholder tekst nemlig 'content'.
Ved brug af modulet 'TfidfVectorizer' kan vi omdanne hver tekst til en række af vægtede tal baseret på TF-IDF (Term Frequency-Inverse Document Frequency).
- Term Frequency (TF): Hvor ofte et ord optræder i en given tekst.
- Inverse Document Frequency (IDF): Hvor sjældent ordet optræder på tværs af alle dokumenter – jo sjældnere, desto vigtigere.

In [51]:
#Vektoriser tekstdata (TF-IDF)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [52]:
print(X_train_vec.shape)


(67869, 348321)


Dette betyder at vi har 67869 tekster og 348321 unikke ord.

In [53]:
print(X_train_vec[0])

  (0, 154757)	0.692729182306506
  (0, 179957)	0.1811974802545005
  (0, 167945)	0.3418501081670975
  (0, 279436)	0.13908135981551245
  (0, 61642)	0.15374907561930365
  (0, 178328)	0.09524172983917649
  (0, 321903)	0.08309029399705253
  (0, 246848)	0.15347423413731567
  (0, 338081)	0.11252975595183769
  (0, 95720)	0.22311495969615974
  (0, 224941)	0.18219722195171578
  (0, 319791)	0.16713534710957997
  (0, 215972)	0.17041804350934242
  (0, 286813)	0.2118149311476041
  (0, 68009)	0.1351946216338627
  (0, 254829)	0.14195269630850751
  (0, 310451)	0.22805968012240363


Forklaring af outputtet når teksten er blevet nummeriseret:
Eksempel på output: 
(0, 2394)	0.09388827930025204

Række (0): Angiver, at vi ser på den første tekst i datasættet.
Kolonne (2394): Hver kolonne repræsenterer en unik funktion (ord) i vokabularet.
Værdi (0.0939 osv.): TF-IDF-vægten – en højere værdi betyder, at ordet er vigtigere i denne tekst i forhold til resten af korpuset.

In [54]:


# Træn SVM-model
model = LinearSVC(class_weight='balanced',C=0.8, dual=False, max_iter=10000)
model.fit(X_train_vec, y_train)

# Forudsig på testdata
y_pred = model.predict(X_test_vec)

# Evaluer præstation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted') 
print(f"Model accuracy: {accuracy * 100:.2f}%")
print(f"Model F1-score: {f1:.2f}")

Model accuracy: 86.33%
Model F1-score: 0.86


In [55]:
# Funktion til at forudsige om en nyhed er fake eller ægte
def predict_news(news_text):
    news_vec = vectorizer.transform([news_text])  # Vektoriser ny tekst
    prediction = model.predict(news_vec)          # Forudsig
    return prediction[0]

#Conspiracy
new_article = """
look like your use ad blocker pleas whitelist disabl abovetopsecretcom adblock tool thank
"""

result = predict_news(new_article)
print(f"Prediction: {result}")

#Reliable
new_article2 ="""editor suppos let iran bomb week review march unit state iran share interest iraq stabil well afghanistan must work togeth contain present iraqi nose dive toward civil war turn requir mutual moratorium side demon side disallow incendiari rhetor supplant prudent foreign policymak doubtless good gener result success dialogu iraq extend larger issu region secur like even motiv iran endors call gulf cooper council persian gulf nuclearweaponsfre zone kaveh l afrasiabi cambridg mass writer former advis iran nuclear negoti team author book iran nuclear program"""
result2 = predict_news(new_article2)
print(f"Prediction: {result2}")

#Political1
new_article3="""convict fraudster former pharmaceut compani ceo martin shkreli dub hate man america head jail bail revok facebook post offer cur strand hillari clinton hair us district judg kiyo matsumoto rule wednesday shkreli"""
print(f"Prediction: {predict_news(new_article3)}")

#Political2
new_article4="""red eye open thread dko list complet hit leadership delay blunt hit swing state hit toptier race knowl stork murphi embrac famili seemann newberri david vs goliath two well morrison target race verg toptier"""
print(f"Prediction: {predict_news(new_article4)}")


Prediction: Fake News
Prediction: Reliable News
Prediction: Fake News
Prediction: Reliable News
